In [1]:
!ls -lh data/

total 757M
-rw-r--r-- 1 RENT 197121 379M ׳“׳¦׳� 11 16:36 fixed.csv
-rw-r--r-- 1 RENT 197121 379M ׳“׳¦׳� 11 15:43 oddetect.csv


# Loading The CSV File

In [117]:
%matplotlib inline
import pandas as pd

In [136]:
df = pd.read_csv('data/fixed.csv',names=["frame", "x", "y", "obj", "size", "seq", "tbd1", "tbd2", "tbd3", "filename", "time", "path_time", "delta_time", "tbd4"], usecols=[ "x", "y", "obj", "size", "seq", "filename", "time", "path_time", "delta_time"],parse_dates=[ 
    "time"])
df

,x,y,obj,size,seq,filename,time,path_time,delta_time
0,14,329,0,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00
1,4,291,1,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00
2,629,7,2,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00
3,253,4,3,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00
4,185,4,4,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00
5,5,102,5,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:28:17,0:01:08
6,6,102,5,362,1,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:28:17,0:01:08
7,6,102,5,362,2,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:28:17,0:01:08
8,7,102,5,362,3,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:28:17,0:01:08
9,9,102,5,362,4,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:28:17,0:01:08


In [137]:
df['time'] = df['time'] + pd.to_timedelta(df['delta_time'])
df['time_hour'] = df.time.dt.time
df['date'] = df.time.dt.date
df = df.drop(['delta_time', 'path_time', 'time'], axis=1)
df

,x,y,obj,size,seq,filename,time_hour,date
0,14,329,0,362,0,14-16\20170817012709_100.h264.txt,01:27:09,2017-08-17
1,4,291,1,362,0,14-16\20170817012709_100.h264.txt,01:27:09,2017-08-17
2,629,7,2,362,0,14-16\20170817012709_100.h264.txt,01:27:09,2017-08-17
3,253,4,3,362,0,14-16\20170817012709_100.h264.txt,01:27:09,2017-08-17
4,185,4,4,362,0,14-16\20170817012709_100.h264.txt,01:27:09,2017-08-17
5,5,102,5,362,0,14-16\20170817012709_100.h264.txt,01:28:17,2017-08-17
6,6,102,5,362,1,14-16\20170817012709_100.h264.txt,01:28:17,2017-08-17
7,6,102,5,362,2,14-16\20170817012709_100.h264.txt,01:28:17,2017-08-17
8,7,102,5,362,3,14-16\20170817012709_100.h264.txt,01:28:17,2017-08-17
9,9,102,5,362,4,14-16\20170817012709_100.h264.txt,01:28:17,2017-08-17


In [121]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997979 entries, 0 to 2997978
Data columns (total 8 columns):
x            int64
y            int64
obj          int64
size         int64
seq          int64
filename     object
time_hour    object
date         object
dtypes: int64(5), object(3)
memory usage: 626.0 MB


In [138]:
df.describe()


,x,y,obj,size,seq
count,2.997979e+06,2.997979e+06,2.997979e+06,2.997979e+06,2.997979e+06
mean,2.730162e+02,1.480890e+02,5.053650e+01,4.675812e+07,1.158957e+02
std,1.609973e+02,7.650773e+01,8.449936e+01,3.391170e+07,1.502103e+02
min,3.000000e+00,3.000000e+00,0.000000e+00,3.620000e+02,0.000000e+00
25%,1.440000e+02,9.500000e+01,1.200000e+01,1.419073e+07,1.600000e+01
50%,2.690000e+02,1.340000e+02,2.800000e+01,4.510348e+07,5.900000e+01
75%,4.020000e+02,2.050000e+02,5.100000e+01,5.399656e+07,1.620000e+02
max,6.370000e+02,3.570000e+02,6.430000e+02,1.265195e+08,1.977000e+03


In [139]:
%%time
df_obj = df.select_dtypes(include = ['object']).copy()
converted_obj = pd.DataFrame()

for col in df_obj.columns:
        num_unique_values = len(df_obj[col].unique())
        num_total_values = len(df_obj[col])
        if num_unique_values / num_total_values < 0.5:
            converted_obj.loc[:,col] = df_obj[col].astype('category')
        else:
            converted_obj.loc[:,col] = df_obj[col]


compare_obj = pd.concat([df_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)
df[converted_obj.columns] = converted_obj


Wall time: 5.25 s


In [124]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997979 entries, 0 to 2997978
Data columns (total 8 columns):
x            int64
y            int64
obj          int64
size         int64
seq          int64
filename     category
time_hour    category
date         category
dtypes: category(3), int64(5)
memory usage: 137.9 MB


In [140]:
df_int = df.select_dtypes(include=['int64'])
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')
df[converted_int.columns] = converted_int

In [141]:
len(df)

2997979

In [127]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997979 entries, 0 to 2997978
Data columns (total 8 columns):
x            uint16
y            uint16
obj          uint16
size         uint32
seq          uint16
filename     category
time_hour    category
date         category
dtypes: category(3), uint16(4), uint32(1)
memory usage: 57.8 MB


In [142]:
%time df.drop_duplicates()

Wall time: 1.43 s


,x,y,obj,size,seq,filename,time_hour,date
0,14,329,0,362,0,14-16\20170817012709_100.h264.txt,01:27:09,2017-08-17
1,4,291,1,362,0,14-16\20170817012709_100.h264.txt,01:27:09,2017-08-17
2,629,7,2,362,0,14-16\20170817012709_100.h264.txt,01:27:09,2017-08-17
3,253,4,3,362,0,14-16\20170817012709_100.h264.txt,01:27:09,2017-08-17
4,185,4,4,362,0,14-16\20170817012709_100.h264.txt,01:27:09,2017-08-17
5,5,102,5,362,0,14-16\20170817012709_100.h264.txt,01:28:17,2017-08-17
6,6,102,5,362,1,14-16\20170817012709_100.h264.txt,01:28:17,2017-08-17
7,6,102,5,362,2,14-16\20170817012709_100.h264.txt,01:28:17,2017-08-17
8,7,102,5,362,3,14-16\20170817012709_100.h264.txt,01:28:17,2017-08-17
9,9,102,5,362,4,14-16\20170817012709_100.h264.txt,01:28:17,2017-08-17


In [129]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997979 entries, 0 to 2997978
Data columns (total 8 columns):
x            uint16
y            uint16
obj          uint16
size         uint32
seq          uint16
filename     category
time_hour    category
date         category
dtypes: category(3), uint16(4), uint32(1)
memory usage: 57.8 MB


In [143]:
%time df.to_pickle('data/to_pickle.pk1.xz')

Wall time: 17.5 s


In [114]:
%time unpickled_df = pd.read_pickle("data/to_pickle.pk1.xz")

Wall time: 1.27 s


In [115]:
unpickled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997979 entries, 0 to 2997978
Data columns (total 7 columns):
x           uint16
y           uint16
obj         uint16
size        uint32
seq         uint16
filename    category
time        datetime64[ns]
dtypes: category(1), datetime64[ns](1), uint16(4), uint32(1)
memory usage: 62.9 MB
